# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Lab 2

In this lab we will explore the AI services present in Azure Foundry. This lab will cover the following services:
- Speech
- Language + Translator
- Vision + Document
- Content Safety

Understanding these services allows us to add more capabilities to our applications.

### Exercise 1 - Speech

The Speech service provides speech-to-text and text-to-speech conversion capabilities with a Speech resource. You can transcribe speech to text with high accuracy, produce natural text-to-speech voices, translate spoken audio, and use speaker recognition during conversations. Create custom voices, add specific words to your base vocabulary, or build your own models. Run Speech anywhere, in the cloud or at the edge in containers. It's easy to enable speech in your applications, tools, and devices with the Speech CLI, Speech SDK, and REST APIs.

Common scenarios for speech usage:

**Caption Generation:** Learn how to synchronize captions with your input audio, apply profanity filters, get partial results, apply customizations, and identify spoken languages for multilingual scenarios.

**Audio Content Creation:** You can use neural voices to make interactions with chatbots and voice assistants more natural and engaging, convert digital texts like e-books into audiobooks, and enhance automotive navigation systems.

**Call Center:** Transcribe calls in real-time or process a batch of calls, remove personally identifiable information, and extract insights like sentiment analysis to assist with your call center use case.

**Language Learning:** Provide pronunciation assessment feedback for language learners, offer real-time transcription support for remote learning conversations, and read educational materials aloud using neural voices.

**Voice Assistants:** Create natural, human-like conversational interfaces for your applications and experiences. The voice assistant feature offers fast and reliable interaction between a device and an assistant implementation.

To perform this exercise, verify that your `.env` file has the following variables filled:
- SPEECH_ENDPOINT
- SPEECH_KEY

After verification, let's start by loading the necessary libraries, initializing the client, and making a call to convert audio to text:

In [ ]:
%pip install azure-cognitiveservices-speech

In [ ]:
# Continuous speech recognition to process all audio, even with initial silence
import os
import azure.cognitiveservices.speech as speechsdk
import time
from dotenv import load_dotenv

load_dotenv()

speech_key = os.getenv('SPEECH_KEY')
speech_region = os.getenv('SPEECH_REGION')
audio_file = '../../../samples/audio001.wav'
if speech_key and speech_region:
    try:
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
        speech_config.speech_recognition_language = "en-US"
        audio_config = speechsdk.audio.AudioConfig(filename=audio_file)
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

        recognized_texts = []
        def recognized_cb(evt):
            if evt.result.text:
                print('Recognized:', evt.result.text)
                recognized_texts.append(evt.result.text)

        speech_recognizer.recognized.connect(recognized_cb)

        print("Starting continuous recognition...")
        speech_recognizer.start_continuous_recognition()

        # Wait for recognition to finish (adjust time according to audio size)
        time.sleep(10)
        speech_recognizer.stop_continuous_recognition()

        print("Recognition completed. Full text:")
        print(' '.join(recognized_texts))
    except Exception as e:
        print(f"Error in continuous recognition: {e}")
else:
    print("Please configure the SPEECH_KEY and SPEECH_REGION environment variables.")

### Exercise 2 - Language + Translator

Integrate natural language into applications, bots, and IoT devices. For example, this service can remove sensitive data, segment long meetings into chapters, analyze health records, and orchestrate conversational bots with your custom intents using factual responses.

This Language service unifies the following Azure AI services previously available separately: Text Analytics, QnA Maker, and LUIS.

Azure AI Foundry allows you to use most of the following service features without needing to write code.

In [ ]:
%pip install azure-ai-textanalytics azure-ai-translation-text

In [ ]:
# Language Analytics Example
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient
from azure.ai.translation.text.models import InputTextItem
import os
from dotenv import load_dotenv

load_dotenv()

# Language Analytics
language_endpoint = os.getenv('LANGUAGE_ENDPOINT')
language_key = os.getenv('LANGUAGE_KEY')

# Translator
translator_endpoint = os.getenv('TRANSLATOR_ENDPOINT')
translator_key = os.getenv('TRANSLATOR_KEY')
translator_region = os.getenv('TRANSLATOR_REGION')

if language_endpoint and language_key:
    try:
        print("=== Azure AI Language Analytics ===")
        
        # Create Language client
        text_analytics_client = TextAnalyticsClient(
            endpoint=language_endpoint,
            credential=AzureKeyCredential(language_key)
        )
        
        # Sample texts for analysis
        documents = [
            "I absolutely love this new product! It's amazing and works perfectly.",
            "This service is terrible and I'm very disappointed with the quality.",
            "The weather today is neutral, neither good nor bad."
        ]
        
        print("\n📊 Sentiment Analysis:")
        # Sentiment analysis
        sentiment_results = text_analytics_client.analyze_sentiment(documents=documents)
        
        for idx, result in enumerate(sentiment_results):
            if not result.is_error:
                print(f"\nDocument {idx + 1}: {documents[idx][:50]}...")
                print(f"Sentiment: {result.sentiment}")
                print(f"Confidence scores - Positive: {result.confidence_scores.positive:.2f}, "
                      f"Negative: {result.confidence_scores.negative:.2f}, "
                      f"Neutral: {result.confidence_scores.neutral:.2f}")
            else:
                print(f"Error in document {idx + 1}: {result.error}")
        
        print("\n🏷️ Key Phrase Extraction:")
        # Key phrase extraction
        key_phrases_results = text_analytics_client.extract_key_phrases(documents=documents)
        
        for idx, result in enumerate(key_phrases_results):
            if not result.is_error:
                print(f"\nDocument {idx + 1}: {documents[idx][:50]}...")
                print(f"Key phrases: {', '.join(result.key_phrases)}")
            else:
                print(f"Error in document {idx + 1}: {result.error}")
                
    except Exception as e:
        print(f"Error in language analytics: {e}")
        
else:
    print("Define LANGUAGE_ENDPOINT and LANGUAGE_KEY variables in the .env file")

# Translator
if translator_key and translator_region:
    try:
        print("\n\n=== Azure AI Translator ===")
        
        # Create Translator client
        translator_client = TextTranslationClient(
            endpoint=translator_endpoint,
            credential=AzureKeyCredential(translator_key),
            region=translator_region
        )
        
        # Text to translate
        source_text = "Hello, world! This is a test of the Azure AI Translator service."
        input_text_elements = [InputTextItem(text=source_text)]
        
        print(f"\n📝 Original text: {source_text}")
        print("\n🌍 Translations:")
        
        # Translate to multiple languages
        target_languages = ['pt', 'es', 'fr', 'de', 'ja']
        
        response = translator_client.translate(
            content=input_text_elements,
            to=target_languages
        )
        
        for translation_result in response:
            for translation in translation_result.translations:
                language_names = {
                    'pt': 'Portuguese',
                    'es': 'Spanish', 
                    'fr': 'French',
                    'de': 'German',
                    'ja': 'Japanese'
                }
                language_name = language_names.get(translation.to, translation.to)
                print(f"  {language_name} ({translation.to}): {translation.text}")
                
    except Exception as e:
        print(f"Error in translator: {e}")
        
else:
    print("Define TRANSLATOR_KEY and TRANSLATOR_REGION variables in the .env file")

### Exercise 3 - Vision + Document

Azure AI Vision is a unified service that offers innovative computer vision capabilities. Give your apps the ability to analyze images, read text, and detect faces with prebuilt image tagging, text extraction with optical character recognition (OCR), and responsible facial recognition. Incorporate vision features into your projects with no machine learning experience required.

**Computer Vision** - Extracts rich information from images to categorize and process visual data.

**Face** - The Face service provides AI algorithms that detect, recognize, and analyze human faces in images. Facial recognition software is important in many different scenarios, such as identification, touchless access control, and face blurring for privacy.

**Video Retrieval** - Video Retrieval allows you to create an index of videos that you can search with natural language.

In [ ]:
%pip install azure-ai-vision-imageanalysis azure-ai-formrecognizer

In [ ]:
import os
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

# Configure Vision client
vision_endpoint = os.getenv('AZURE_VISION_ENDPOINT')
vision_key = os.getenv('AZURE_VISION_KEY')

if vision_endpoint and vision_key:
    try:
        print("=== Azure AI Vision - Image Analysis ===")
        
        # Create client
        client = ImageAnalysisClient(
            endpoint=vision_endpoint,
            credential=AzureKeyCredential(vision_key)
        )
        
        # Analyze remote image
        print("\n1. Analyzing remote image...")
        result = client.analyze_from_url(
            image_url="https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png",
            visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ],
            gender_neutral_caption=True,  # Optional (default is False)
        )
        
        print("Analysis results:")
        
        # Display caption results
        print("\n📝 Caption:")
        if result.caption is not None:
            print(f"   '{result.caption.text}', Confidence: {result.caption.confidence:.4f}")
        else:
            print("   No caption found")

        # Display OCR results (extracted text)
        print("\n📖 Extracted text (OCR):")
        if result.read is not None:
            for block in result.read.blocks:
                for line in block.lines:
                    print(f"   Line: '{line.text}', Bounding box: {line.bounding_polygon}")
                    for word in line.words:
                        print(f"     Word: '{word.text}', Confidence: {word.confidence:.4f}")
        else:
            print("   No text found")
            
    except Exception as e:
        print(f"Error in image analysis: {e}")
        
else:
    print("Define AZURE_VISION_ENDPOINT and AZURE_VISION_KEY variables in the .env file")

In [ ]:
# Advanced example - Complete local image analysis
if vision_endpoint and vision_key:
    try:
        print("\n=== Advanced Image Analysis ===")
        
        # Analyze local image
        image_path = "../../../samples/car-accident.png"
        
        print(f"\n2. Analyzing local image: {image_path}")
        
        # Open local image
        with open(image_path, "rb") as image_data:
            # Complete analysis with multiple features
            result = client.analyze(
                image_data=image_data.read(),
                visual_features=[
                    VisualFeatures.CAPTION,
                    VisualFeatures.READ,
                    VisualFeatures.TAGS,
                    VisualFeatures.OBJECTS,
                    VisualFeatures.PEOPLE,
                    VisualFeatures.SMART_CROPS
                ],
                gender_neutral_caption=True
            )
            
            print("\n📝 Caption:")
            if result.caption:
                print(f"   '{result.caption.text}', Confidence: {result.caption.confidence:.4f}")
            
            print("\n🏷️ Identified tags:")
            if result.tags:
                for tag in result.tags.list:
                    print(f"   - {tag.name}: {tag.confidence:.4f}")
            
            print("\n📦 Detected objects:")
            if result.objects:
                for obj in result.objects.list:
                    print(f"   - {obj.tags[0].name}: {obj.tags[0].confidence:.4f}")
                    print(f"     Location: {obj.bounding_box}")
            
            print("\n👥 Detected people:")
            if result.people:
                for person in result.people.list:
                    print(f"   - Person detected with confidence: {person.confidence:.4f}")
                    print(f"     Location: {person.bounding_box}")
            
            print("\n✂️ Smart crops:")
            if result.smart_crops:
                for crop in result.smart_crops.list:
                    print(f"   - Aspect {crop.aspect_ratio}: {crop.bounding_box}")
                    
    except FileNotFoundError:
        print(f"Image file not found: {image_path}")
        print("Make sure the file exists at the specified path.")
    except Exception as e:
        print(f"Error in advanced analysis: {e}")

In [ ]:
# Example - Document Intelligence
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# Configure Document Intelligence client
doc_intelligence_endpoint = os.getenv('DOC_INTELLIGENCE_ENDPOINT')
doc_intelligence_key = os.getenv('DOC_INTELLIGENCE_KEY')

if doc_intelligence_endpoint and doc_intelligence_key:
    try:
        print("\n=== Document Intelligence ===")
        
        # Create client
        document_analysis_client = DocumentAnalysisClient(
            endpoint=doc_intelligence_endpoint,
            credential=AzureKeyCredential(doc_intelligence_key)
        )
        
        # Analyze generic document
        doc_path = "../../../samples/placa.jpg"
        
        with open(doc_path, "rb") as f:
            # Use pre-built model for general layout
            poller = document_analysis_client.begin_analyze_document(
                "prebuilt-layout", document=f
            )
            
        result = poller.result()
        
        print("Document Layout Analysis:")
        print(f"Number of pages: {len(result.pages)}")
        
        # Extract tables
        if result.tables:
            print(f"\nTables found: {len(result.tables)}")
            for idx, table in enumerate(result.tables):
                print(f"Table {idx + 1}: {table.row_count} rows x {table.column_count} columns")
                
        # Extract paragraphs
        if result.paragraphs:
            print(f"\nParagraphs found: {len(result.paragraphs)}")
            for idx, paragraph in enumerate(result.paragraphs[:3]):  # Show only first 3
                print(f"Paragraph {idx + 1}: {paragraph.content[:100]}...")
                
        # Extract key-value pairs
        if result.key_value_pairs:
            print(f"\nKey-value pairs found: {len(result.key_value_pairs)}")
            for kv_pair in result.key_value_pairs[:5]:  # Show only first 5
                if kv_pair.key and kv_pair.value:
                    print(f"  {kv_pair.key.content}: {kv_pair.value.content}")
                    
    except FileNotFoundError:
        print(f"Document file not found: {doc_path}")
    except Exception as e:
        print(f"Error in Document Intelligence: {e}")
        
else:
    print("Define DOC_INTELLIGENCE_ENDPOINT and DOC_INTELLIGENCE_KEY variables in the .env file")

### Exercise 4 - Content Safety

Azure AI Content Safety detects harmful user-generated and AI-generated content in applications and services. This service provides several different types of analysis.

**Prompt Shields** - Examines text for risks of user input attacks on a Large Language Model.

**Groundedness Detection (preview)** - Detects whether text responses from large language models (LLMs) are grounded in the source materials provided by users.

**Protected Material Detection in Text** - Examines AI-generated text for known text content (e.g., song lyrics, articles, recipes, selected web content).

**Custom Categories API (standard) (preview)** - Allows you to create and train your own custom content categories and examine text for matches.

**Custom Categories API (rapid) (preview)** - Allows you to define emerging patterns of harmful content and examine text and images for matches.

**Text Analysis API** - Examines text for sexual, violence, hate, and self-harm content with multiple severity levels.

**Image Analysis API** - Examines images for sexual, violence, hate, and self-harm content with multiple severity levels.

In [ ]:
%pip install azure-ai-contentsafety

In [ ]:
# Example - Content Safety (following official quickstart)
from azure.ai.contentsafety import ContentSafetyClient
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

def analyze_text_content(text_to_analyze):
    """
    Function to analyze text using Azure Content Safety
    Based on Microsoft's official quickstart
    """
    # Get credentials from environment variables
    key = os.getenv('CONTENT_SAFETY_KEY')
    endpoint = os.getenv('CONTENT_SAFETY_ENDPOINT')
    
    if not key or not endpoint:
        print("❌ Error: Define CONTENT_SAFETY_KEY and CONTENT_SAFETY_ENDPOINT variables in the .env file")
        return None
    
    try:
        # Create Azure AI Content Safety client
        client = ContentSafetyClient(endpoint, AzureKeyCredential(key))
        
        # Configure request
        request = AnalyzeTextOptions(text=text_to_analyze)
        
        # Analyze text
        response = client.analyze_text(request)
        
        # Extract results by specific category (following quickstart)
        hate_result = next((item for item in response.categories_analysis if item.category == TextCategory.HATE), None)
        self_harm_result = next((item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM), None)
        sexual_result = next((item for item in response.categories_analysis if item.category == TextCategory.SEXUAL), None)
        violence_result = next((item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE), None)
        
        # Display results
        print(f"📝 Analyzed text: '{text_to_analyze}'")
        print("🔍 Analysis results:")
        
        if hate_result:
            print(f"  💬 Hate: Severity {hate_result.severity}")
        if self_harm_result:
            print(f"  🩹 Self-harm: Severity {self_harm_result.severity}")
        if sexual_result:
            print(f"  🔞 Sexual: Severity {sexual_result.severity}")
        if violence_result:
            print(f"  ⚔️ Violence: Severity {violence_result.severity}")
        
        # Interpret overall risk level
        max_severity = max([
            hate_result.severity if hate_result else 0,
            self_harm_result.severity if self_harm_result else 0,
            sexual_result.severity if sexual_result else 0,
            violence_result.severity if violence_result else 0
        ])
        
        if max_severity == 0:
            risk_level = "✅ Safe"
        elif max_severity <= 2:
            risk_level = "⚠️ Low risk"
        elif max_severity <= 4:
            risk_level = "🔸 Moderate risk"
        else:
            risk_level = "🔴 High risk"
            
        print(f"📊 Overall assessment: {risk_level}")
        return response
        
    except HttpResponseError as e:
        print("❌ Text analysis failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Configure Content Safety client
content_safety_endpoint = os.getenv('CONTENT_SAFETY_ENDPOINT')
content_safety_key = os.getenv('CONTENT_SAFETY_KEY')

if content_safety_endpoint and content_safety_key:
    print("=== Content Safety - Text Analysis ===")
    print("Based on Microsoft's official quickstart")
    print("=" * 50)
    
    # Sample texts for analysis (including different risk levels)
    test_texts = [
        "Hello! How are you today? Have a great day!",
        "This is a neutral text about technology and Python programming.",
        "I'm very angry about this situation, but I'll resolve it in a civilized manner.",
        "Test text for potentially problematic content moderation."
    ]
    
    print(f"📋 Analyzing {len(test_texts)} sample texts...\n")
    
    for idx, text in enumerate(test_texts, 1):
        print(f"🔍 Analysis {idx}/{len(test_texts)}:")
        result = analyze_text_content(text)
        
        if result:
            print("✅ Analysis completed successfully")
        else:
            print("❌ Analysis failed")
            
        print("-" * 50)
        
else:
    print("❌ Required configuration:")
    print("Define CONTENT_SAFETY_ENDPOINT and CONTENT_SAFETY_KEY variables in the .env file")
    print("\nConfiguration example:")
    print("CONTENT_SAFETY_ENDPOINT=https://your-content-safety.cognitiveservices.azure.com/")
    print("CONTENT_SAFETY_KEY=your-content-safety-key")